In [54]:
import os
import openai
import praw

# Import and pass api key
with open(r'C:\Users\mhuh22\Desktop\openai_api.txt', 'r') as file:
    api_key = file.readline().strip()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Import reddit client id and secret
with open(r'C:\Users\mhuh22\Desktop\reddit_api.txt', 'r') as file:
    lines = file.readlines()

client_id = lines[0].strip().split(':')[1]
client_secret = lines[1].strip().split(':')[1]

# Call reddit api
reddit = praw.Reddit(client_id=client_id, 
                     client_secret=client_secret, 
                     user_agent='sentiment analysis test')

In [26]:
# Access the most popular titles in finance
for submission in reddit.subreddit('stocks').hot(limit=5):
    print(submission.title)

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
r/Stocks Daily Discussion Wednesday - Apr 05, 2023
Private payrolls rose by 145,000 in March, well below expectations, ADP says
Anyone Buying $SCHW Heading Into Earnings April 17th?
Apple’s “Wearables & Accessories” revenue ($41.24B) is significantly greater than Nvidia’s annual revenue (26.91B).


In [27]:
subreddit_stocks = reddit.subreddit('stocks')

In [31]:
# Display name
print(subreddit_stocks.display_name)
# Title
print(subreddit_stocks.title)

stocks
Stocks - Investing and trading for all


In [32]:
# View how many redditors are active currently
subreddit_stocks.accounts_active

4872

In [34]:
# View post ids and titles
for post in subreddit_stocks.hot(limit=5):
    print(post.id, post.title)

11ezwar Rate My Portfolio - r/Stocks Quarterly Thread March 2023
12cek09 r/Stocks Daily Discussion Wednesday - Apr 05, 2023
12cmpjk Private payrolls rose by 145,000 in March, well below expectations, ADP says
12cx5hf Anyone Buying $SCHW Heading Into Earnings April 17th?
12buzi8 Apple’s “Wearables & Accessories” revenue ($41.24B) is significantly greater than Nvidia’s annual revenue (26.91B).


In [36]:
# Grab top 2 comments for top 5 posts
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    
    # Grabs replies from post
    submission = reddit.submission(post.id)
    
    # print top 2 comments per title sub
    counter = 0
    for comment in submission.comments:
        if comment.body == '[deleted]': # ignore if post was deleted
            pass
        
        print(comment.body)
        counter += 1
        
        if counter==2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much. I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if I see an undervalued stock I like, I don't hesitate to buy more and when I make my regular contributions, I try weigh towards stocks that might have been beaten down or seem undervalued. My last review in one of these was [Dec 2021

In [50]:
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit=10, num_comments=2, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for c, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):
        
        if c < skip_first:
            continue
        
        c+=(1-skip_first)
        
        titles_and_comments[c] = ""

        submission = reddit.submission(post.id)
        title = post.title
        
        titles_and_comments[c] += "Title: " + title  + "\n\n"
        titles_and_comments[c] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[c] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break

    return titles_and_comments

In [51]:
title_and_comments = get_titles_and_comments()

In [52]:
title_and_comments

{1: 'Title: Private payrolls rose by 145,000 in March, well below expectations, ADP says\n\nComments: \n\nFor whatever it’s worth, ADP is pretty far off in their models often times. They had 242k in Feb, actual 311k. ADP in Jan said 106k, actual 517k.  \n    \nThis isn’t official data. That’s the BLS report, which comes out Friday.\nThese reports always seem questionable. The deceleration of payrolls is not lost on me, but when actual figures come in below an *upwardly revised* estimate, does that not shine a light on faulty projections?\n',
 2: 'Title: Anyone Buying $SCHW Heading Into Earnings April 17th?\n\nComments: \n\nReddit loves SCHW rn. for that reason, I’m out.\nBought 500 shares at a $55 cost basis. \n\nIn Q4 they cited liquidity concerns and took a $13B loan from FHLB because of the bond portfolio blowing up. They still went ahead and raised the dividend 22%, and did $3B worth of share buybacks. \n\nThe actions speak, complemented by the company’s well reputable history. The

In [46]:
def create_prompt(title_and_comments):
    task = """
    Return the stock ticker or company name mentioned in the following title 
    and comments and classify the sentiment around the company as positive, 
    negative, or neutral. If no ticker or company is mentioned write 
    No company mentioned\n\n'      
    """
    return task + title_and_comments

In [49]:
print(title_and_comments[1])

Title: Private payrolls rose by 145,000 in March, well below expectations, ADP says

Comments: 

For whatever it’s worth, ADP is pretty far off in their models often times. They had 242k in Feb, actual 311k. ADP in Jan said 106k, actual 517k.  
    
This isn’t official data. That’s the BLS report, which comes out Friday.
These reports always seem questionable. The deceleration of payrolls is not lost on me, but when actual figures come in below an *upwardly revised* estimate, does that not shine a light on faulty projections?
The market may have wanted employment high, but the Fed doesn't. When the Fed gets what they want, they will stop rate increases, which since the start of xovid seems like the major and obvious driving force of the market as the federal interest rate was lowered than raised.



In [57]:
for key, title_with_comments in title_and_comments.items():
    prompt = create_prompt(title_with_comments)
    response = openai.Completion.create(engine='text-davinci-003',
                                       prompt=prompt,
                                       max_tokens=256,
                                       temperature=0,
                                       top_p=1)
    print(title_with_comments)
    print(f"Sentiment Report from OpenAI: {response['choices'][0]['text']}")
    print('-----------------------')

Title: Private payrolls rose by 145,000 in March, well below expectations, ADP says

Comments: 

For whatever it’s worth, ADP is pretty far off in their models often times. They had 242k in Feb, actual 311k. ADP in Jan said 106k, actual 517k.  
    
This isn’t official data. That’s the BLS report, which comes out Friday.
These reports always seem questionable. The deceleration of payrolls is not lost on me, but when actual figures come in below an *upwardly revised* estimate, does that not shine a light on faulty projections?

Sentiment Report from OpenAI: 
No company mentioned
-----------------------
Title: Anyone Buying $SCHW Heading Into Earnings April 17th?

Comments: 

Reddit loves SCHW rn. for that reason, I’m out.
Bought 500 shares at a $55 cost basis. 

In Q4 they cited liquidity concerns and took a $13B loan from FHLB because of the bond portfolio blowing up. They still went ahead and raised the dividend 22%, and did $3B worth of share buybacks. 

The actions speak, complement